In [1]:
import numpy as np
import pandas as pd
import tflearn

In [2]:
train = pd.read_csv("final_train.csv")
test = pd.read_csv("final_test.csv")

In [3]:
yID = test["ID"]

In [4]:
train = train.drop(train.columns[[0,1]],axis=1)

In [5]:
test = test.drop(test.columns[[0,1]],axis=1)

In [6]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,12,11,12,15,13,10,14,45,66,84,...,106,108,116,136,141,118,98,94,89,78
1,53,38,25,25,35,80,127,123,128,122,...,31,30,29,28,27,23,24,22,26,29
2,10,7,5,7,5,3,7,0,10,13,...,162,160,158,159,153,150,147,145,142,140
3,24,35,25,13,31,58,79,105,106,103,...,69,44,27,45,83,109,103,74,65,60
4,95,94,80,100,108,134,141,160,166,175,...,147,134,132,145,155,180,169,122,113,110


In [7]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,Class
0,82,220,197,168,162,162,198,191,133,95,...,49,52,59,73,95,122,145,172,180,MIDDLE
1,27,34,60,73,77,89,102,100,106,101,...,57,50,43,43,43,43,50,51,58,OLD
2,37,22,18,15,24,35,32,42,44,42,...,49,46,39,40,36,13,40,54,122,MIDDLE
3,15,21,38,36,42,45,40,38,18,18,...,89,96,92,85,78,75,70,51,34,MIDDLE
4,84,75,78,80,80,91,104,123,130,144,...,92,82,92,101,98,101,98,133,145,YOUNG


In [8]:
nouts = 3
ninps = 784

In [9]:
xtrain = np.array(train.iloc[:,0:784])
#print(np.shape(xtrain))
ytrain = train.iloc[:,784]
#np.shape(ytrain)

In [10]:
ytrain = np.array(pd.get_dummies(ytrain))


In [20]:
ytrain[4]

array([0, 0, 1], dtype=uint8)

In [12]:
input_layer = tflearn.input_data(shape=[None,784])
hl1 = tflearn.fully_connected(input_layer,64,'relu')
hl2 = tflearn.fully_connected(hl1,64,activation='relu')
out = tflearn.fully_connected(hl2,3,activation="sigmoid")

In [13]:
sgd = tflearn.SGD(learning_rate=0.01,lr_decay=0.001,decay_step=500)

In [14]:
net = tflearn.regression(out,optimizer=sgd,loss = "categorical_crossentropy")

In [15]:
model = tflearn.DNN(net)

In [16]:
model.fit(xtrain,ytrain,n_epoch=25,batch_size= 100,show_metric=True)

Training Step: 4999  | total loss: 0.85745 | time: 1.239s
| SGD | epoch: 025 | loss: 0.85745 - acc: 0.5881 -- iter: 19900/19906
Training Step: 5000  | total loss: 0.84867 | time: 1.245s
| SGD | epoch: 025 | loss: 0.84867 - acc: 0.5943 -- iter: 19906/19906
--


In [28]:
preds = model.predict(test)

In [29]:
classes = []
for i in range(len(test)):
    if np.argmax(preds[i]) == 0:
        classes.append("MIDDLE")
    elif np.argmax(preds[i]) == 1:
        classes.append("OLD")
    else:
        classes.append("YOUNG")

In [30]:
df = pd.DataFrame({"Class": classes,"ID":yID})

In [31]:
df.head()

,Class,ID
0,MIDDLE,10.jpg
1,MIDDLE,1000.jpg
2,MIDDLE,10002.jpg
3,MIDDLE,10003.jpg
4,MIDDLE,10005.jpg


In [32]:
df.to_csv("submission1.csv",index=False)